In [ ]:
-- This query combines and compares the average HP, Attack, and Defense stats across the three Pokémon generations.

WITH CombinedStats AS (
    -- Selects the average HP, Attack, and Defense stats from Generation 1
    SELECT 'Gen1' AS Generation, AVG(HP) AS AvgHP, AVG(Attack) AS AvgAttack, AVG(Defense) AS AvgDefense
    FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Selects the average HP, Attack, and Defense stats from Generation 2
    SELECT 'Gen2', AVG(HP), AVG(Attack), AVG(Defense)
    FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Selects the average HP, Attack, and Defense stats from Generation 3
    SELECT 'Gen3', AVG(HP), AVG(Attack), AVG(Defense)
    FROM PokemonGen3.dbo.PokemonGen3
)
-- The final SELECT fetches the combined average stats for each generation
SELECT Generation, AvgHP, AvgAttack, AvgDefense
FROM CombinedStats;

In [ ]:
-- This query finds the most common Pokémon type across all three generations.

SELECT TOP 1 Type, COUNT(*) AS TypeCount
FROM (
    -- Include all 'Type1' from Gen1
    SELECT Type1 AS Type FROM PokemonGen1.dbo.PokemonGen1 WHERE Type1 IS NOT NULL
    UNION ALL
    -- Include all 'Type2' from Gen1
    SELECT Type2 AS Type FROM PokemonGen1.dbo.PokemonGen1 WHERE Type2 IS NOT NULL
    UNION ALL
    -- Include all 'Type1' from Gen2
    SELECT Type1 AS Type FROM PokemonGen2.dbo.PokemonGen2 WHERE Type1 IS NOT NULL
    UNION ALL
    -- Include all 'Type2' from Gen2
    SELECT Type2 AS Type FROM PokemonGen2.dbo.PokemonGen2 WHERE Type2 IS NOT NULL
    UNION ALL
    -- Include all 'Type1' from Gen3
    SELECT Type1 AS Type FROM PokemonGen3.dbo.PokemonGen3 WHERE Type1 IS NOT NULL
    UNION ALL
    -- Include all 'Type2' from Gen3
    SELECT Type2 AS Type FROM PokemonGen3.dbo.PokemonGen3 WHERE Type2 IS NOT NULL
) AS CombinedTypes
GROUP BY Type
ORDER BY TypeCount DESC;


In [ ]:
-- This query identifies the Pokémon with the highest total stats (sum of all individual stats) across all generations.

SELECT MAX(CombinedTotal.Total) AS MaxStat, CombinedTotal.Name, CombinedTotal.Generation
FROM (
    -- This subquery calculates the total stats for each Pokémon in Generation 1 and labels them 'Gen1'.
    SELECT Name, (HP + Attack + Defense + SpecialAttack + SpecialDefense + Speed) AS Total, 'Gen1' AS Generation FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Similarly, it calculates the total stats for each Pokémon in Generation 2 and labels them 'Gen2'.
    SELECT Name, (HP + Attack + Defense + SpecialAttack + SpecialDefense + Speed), 'Gen2' FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- And does the same for Generation 3, labeling them 'Gen3'.
    SELECT Name, (HP + Attack + Defense + SpecialAttack + SpecialDefense + Speed), 'Gen3' FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedTotal
-- Grouping the results by Pokémon name and generation.
GROUP BY CombinedTotal.Name, CombinedTotal.Generation;



In [ ]:
-- This query calculates the average speed of Pokémon based on their type classification (Single or Dual) for each generation.

SELECT Generation, TypeCategory, AVG(Speed) AS AverageSpeed
FROM (
    -- Create a subquery that selects generation, type category, and speed for Gen1 Pokémon
    SELECT 'Gen1' AS Generation, 
           CASE WHEN Type2 IS NOT NULL THEN 'Dual' ELSE 'Single' END AS TypeCategory, 
           Speed 
    FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Repeat the same selection for Gen2 Pokémon, classifying type category based on the presence of Type2
    SELECT 'Gen2', 
           CASE WHEN Type2 IS NOT NULL THEN 'Dual' ELSE 'Single' END, 
           Speed 
    FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Repeat the selection for Gen3 Pokémon, continuing the type category classification
    SELECT 'Gen3', 
           CASE WHEN Type2 IS NOT NULL THEN 'Dual' ELSE 'Single' END, 
           Speed 
    FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedTypes
-- Group the results by generation and type category
GROUP BY Generation, TypeCategory;



In [ ]:
-- This query calculates the number of legendary Pokémon in each generation.
SELECT Generation, COUNT(*) AS LegendaryCount
FROM (
    -- Selects all Pokémon from Generation 1 that have the 'Pressure' ability, which is assumed to be a marker of legendary Pokémon.
    SELECT 'Gen1' AS Generation, Name 
    FROM PokemonGen1.dbo.PokemonGen1 
    WHERE Ability1 = 'Pressure' OR Ability2 = 'Pressure' OR Ability3 = 'Pressure'
    UNION ALL
    -- Selects all Pokémon from Generation 2 with the 'Pressure' ability.
    SELECT 'Gen2', Name 
    FROM PokemonGen2.dbo.PokemonGen2 
    WHERE Ability1 = 'Pressure' OR Ability2 = 'Pressure' OR Ability3 = 'Pressure'
    UNION ALL
    -- Selects all Pokémon from Generation 3 with the 'Pressure' ability.
    SELECT 'Gen3', Name 
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Ability1 = 'Pressure' OR Ability2 = 'Pressure' OR Ability3 = 'Pressure'
) AS LegendaryPokemons -- The subquery results are aliased as LegendaryPokemons.
GROUP BY Generation -- Groups the results by generation to prepare for the count.
-- The count function is used to count the number of legendary Pokémon in each group.


In [ ]:
-- This query lists the Pokémon from all generations ordered by their Defense to Attack ratio in descending order.

-- Create a virtual table that combines the Name, Defense, and Attack stats for all Pokémon across generations.
SELECT Generation, Name, Defense, Attack, CAST(Defense AS FLOAT) / Attack AS DefenseAttackRatio
FROM (
    -- Select Name, Defense, and Attack stats from Generation 1 Pokémon.
    SELECT 'Gen1' AS Generation, Name, Defense, Attack FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Repeat the selection for Generation 2.
    SELECT 'Gen2', Name, Defense, Attack FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Repeat the selection for Generation 3.
    SELECT 'Gen3', Name, Defense, Attack FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedStats
-- Order the results by the Defense to Attack ratio, from highest to lowest.
ORDER BY DefenseAttackRatio DESC;